In [ ]:
!git clone 'https://github.com/LuchnikovI/Dynamic-mode-decomposition-for-open-quantum-systems-identification'
%cd 'Dynamic-mode-decomposition-for-open-quantum-systems-identification'
import tensorflow as tf
import dmd

In [3]:
# random X tensor
X_re = tf.random.normal((100, 100, 100), dtype=tf.float64)
X_im = tf.random.normal((100, 100, 100), dtype=tf.float64)
X = tf.complex(X_re, X_im)

# random Y tensor
Y_re = tf.random.normal((100, 100, 100), dtype=tf.float64)
Y_im = tf.random.normal((100, 100, 100), dtype=tf.float64)
Y = tf.complex(Y_re, Y_im)

# dmd based transition matrix
lmbd, right, left = dmd.dmd(X, Y)
T_dmd = (right * lmbd) @ tf.linalg.adjoint(left)

# pinv based transition matrix
T = dmd.solve_regression(X, Y)

# relative difference
delta = tf.linalg.norm(T - T_dmd) / tf.linalg.norm(T + T_dmd)
print(tf.abs(delta).numpy())

1.2042338959885801e-14
